# Time Range Annotation

## Requirements
- [ ] Add/remove individual annotations: 
  - 


In [ ]:
import numpy as np; np.random.seed(0)
import pandas as pd
from scipy.stats import zscore
import string

import colorcet as cc
import holoviews as hv; hv.extension('bokeh')
from holoviews.plotting.links import RangeToolLink
from holoviews.operation.datashader import rasterize
from holoviews import Dataset
from bokeh.models import HoverTool
from holonote.annotate import Annotator
from holonote.app import PanelWidgets
import panel as pn
pn.extension('tabulator')

## Create Multi-Channel Timeseries Plot

We'll use the the [small-data approach](./small_multi-chan-ts.ipynb), but the annotations should work with any of the demonstrated approaches.

### Generate fake data

In [ ]:
n_channels = 8
n_seconds = 300
fs = 256  # Sampling frequency

init_freq = .01  # Initial sine wave frequency in Hz
freq_inc = 2/n_channels  # Frequency increment
amplitude = 1

total_samples = n_seconds * fs
time = np.linspace(0, n_seconds, total_samples)
channels = [f'CH {i}' for i in range(n_channels)]
groups = ['EEG'] * (n_channels // 2) + ['MEG'] * (n_channels - n_channels // 2)

data = np.array([amplitude * np.sin(2 * np.pi * (init_freq + i * freq_inc) * time)
                 for i in range(n_channels)])
print(f'shape: {data.shape} (n_channels, samples) ')

### Create Plot

- TODO: why does including the minimap limit the height of the subcoord plot?

In [ ]:
time_dim = hv.Dimension('Time', unit='s')
amplitude_dim = hv.Dimension('Amplitude', unit='µV')

curves = []
for group, channel, channel_data in zip(groups, channels, data):
    ds = Dataset((time, channel_data), [time_dim, amplitude_dim])
    curve = hv.Curve(ds, time_dim, amplitude_dim, group=group, label=f'{channel}')
    curve.opts(
        subcoordinate_y=True,
        subcoordinate_scale=.75,
        color="black",
        line_width=1,
        tools=['hover'],
        hover_tooltips=[("Group", "$group"), ("Channel", "$label"), "Time", "Amplitude"],
        apply_hard_bounds=True,
        )
    curves.append(curve)

curves_overlay = hv.Overlay(curves, kdims="Channel")

curves_overlay = curves_overlay.opts(
    xlabel="Time (s)",
    ylabel="Channel",
    show_legend=False,
    padding=0,
    responsive=True,
    shared_axes=False,
)

# Minimap
y_positions = range(len(channels))
yticks = [(i, ich) for i, ich in enumerate(channels)]
z_data = zscore(data, axis=1)
minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))
minimap = minimap.opts(
    cmap="RdBu_r",
    colorbar=False,
    xlabel='',
    alpha=0.5,
    yticks=[yticks[0], yticks[-1]],
    max_height=150,
    responsive=True,
    default_tools=[],
)

# Link the curves plot to minimap
link = RangeToolLink(minimap, curves_overlay, axes=["x", "y"],
              boundsy=(-.5, 3.5),
              boundsx=(0, time[len(time)//3])
             )

pn.Column((curves_overlay + minimap).cols(1), min_height=500)


## Time-Range Annotation

### Create fake time range annotations

In [ ]:
def create_range_annotations(n_total_seconds: int, n_categories: int, 
                             n_total_annotations: int, duration: int = 1) -> pd.DataFrame:

    
    start_times = np.sort(np.random.randint(0, n_total_seconds - duration, n_total_annotations))
    
    # Ensure the annotations are non-overlapping
    for i in range(1, len(start_times)):
        if start_times[i] < start_times[i-1] + duration:
            start_times[i] = start_times[i-1] + duration
    end_times = start_times + duration
    categories = np.random.choice(list(string.ascii_uppercase)[:n_categories], n_total_annotations)
    
    df = pd.DataFrame({
        'start': start_times,
        'end': end_times,
        'category': categories
    })
    df['category'] = df['category'].astype('category')
    df = df.sort_values('start')
    return df

np.random.seed(3)
n_categories = 4
n_total_annotations = 50
annotations_df = create_range_annotations(n_seconds, n_categories, n_total_annotations)
unique_categories = list(annotations_df['category'].unique()) + ['other']
unique_categories.sort()
print(unique_categories)
annotations_df.head()

## Create and Populate Annotator Instance

First, define the Annotator instance. We'll specify the primary annotation 'Time' range type as float and then list the field we want to include, 'category'

In [ ]:
annotator = Annotator({"Time": float}, fields=["category"])
print('spec:', annotator.spec)
print('fields:', annotator.fields)

Next, we'll add our fake annotations to the annotator table

In [ ]:
annotator.define_annotations(annotations_df, Time=("start", "end"))
annotator.df.head()

## Define styling

TODO: This is not intuitive.. it should just be done automatically and hidden from most users

In [ ]:
color_map = dict(zip(unique_categories, cc.glasbey[:len(unique_categories)]))
annotator.style.color = hv.dim("category").categorize(categories=color_map, default="grey")
annotator.style.color

In [ ]:
color_map

In [ ]:
styles = ":root {"
class_names = []
for index, category in enumerate(unique_categories, start=1):
    class_name = f"custom-color-{index}"
    class_names.append(class_name)
    color = color_map.get(category, '#333')  # Default color if not found
    styles += f"""
    --{class_name}-bg-color: {color};
    --{class_name}-border-color: {color};
    """
styles += "}"

# CSS rules using the custom properties
button_css = "\n".join(f"""
:host(.{class_name}) .bk-btn {{
    background-color: var(--{class_name}-bg-color) !important;
    border-color: var(--{class_name}-border-color) !important;
}}
""" for class_name in class_names)
styles += button_css

In [ ]:
styles

TODO: Why was this groupby needed? It's not intuitive and doesn't seem to make an impact?

In [ ]:
# annotator.groupby = "category"
# annotator.groupby

## Create Widgets

- TODO: There should be a default API for something like `annotator.widget_select` to create all this automatically
- TODO: make the table responsive to any updates by the GUI, like adding or subtracting annotations
- TODO: adding the annotator tools to the servable app prevents anything from displaying when served
- TODO: Why can't I insert floats into the table cols of start, end in the GUI?
- TODO: what does the pencil icon do?
- TODO: how to color each button of a checkbuttongroup so I don't have to do all this

In [ ]:
colored_widgets_select = [
    pn.widgets.Toggle(name=cat,
                      button_style='outline',
                      stylesheets=[f':host {{ --surface-color: {clr}; }}'],
                      margin=(0, 0, 0, 10 if i == 0 else 0))
    for i, (cat, clr) in enumerate(color_map.items()) 
]
colored_widgets_select = pn.Row(*colored_widgets_select)

In [ ]:
widget_select = pn.widgets.CheckButtonGroup(value=unique_categories, options=unique_categories,)
annotator.visible = widget_select
annotator_tools = pn.panel(PanelWidgets(annotator, {"category": unique_categories}))

widgets = pn.WidgetBox(
    pn.pane.Markdown('## Time Annotation', align='center'),
    pn.pane.Markdown('### Show/Hide'),
    widget_select,
    colored_widgets_select,
    pn.layout.Divider(),
    # pn.pane.Markdown('### Table'),
    pn.Row(pn.widgets.Select(name='Select all of', value=unique_categories[0], options=unique_categories, width=100),
    pn.widgets.Button(name='Deselect',
                     icon='deselect',
                     button_type='primary',
                      align='end'),
    pn.widgets.Button(name='Delete',
                     icon='skull',
                     button_type='danger',
                      align='end')),
    pn.widgets.Tabulator(annotator.df.rename(columns={'start[Time]': 'start', 'end[Time]': 'end'}).reset_index(drop=True),
                         pagination='local',
                         page_size=10,
                         layout='fit_data_fill',
                         editors={'index': None,
                                  'category': {'type': 'list', 'valuesLookup': True},
                                  'start': {'type': 'float'},
                                  'end': {'type': 'float'},
                                 },
                        ),
    pn.layout.Divider(),
    pn.pane.Markdown('### Create New'),
    pn.widgets.Select(name='category', value=unique_categories[0], options=unique_categories,),
    pn.Row(pn.widgets.FloatInput(name='start', width=80),
           pn.widgets.FloatInput(name='end', width=80),
           pn.widgets.Button(name='Add',
                             width=80,
                             align='end',
                             icon='hand-click',
                             button_type='primary')
          ),
    pn.layout.Divider(),
    pn.pane.Markdown('### Save or Revert'),
    pn.Row(pn.widgets.Button(name='Save Changes',
                 icon='device-floppy',
                 button_type='success',
                  align='end'),
    pn.widgets.Button(name='Revert Changes',
             icon='arrow-back-up',
             button_type='warning',
              align='end')
          ),
    pn.layout.Divider(),
    pn.pane.Markdown('### Old'),
    annotator_tools,
    margin=(40,10),
)

## Display Widgets with Plot

- TODO: the rangelink tool doesn't work in the y-dim when annotations are overlaid
- TODO: The color of newly created annotations should match the active category prior to 'apply changes'
- TODO: This should be automatic.. we should have to extract something from the annotator to then overlay.. you should be able to just overlay it like app * annotator.
- TODO: Why do we have to re-apply some the opts to the new composite curves overlay but not the minimap? Seems like a bug
- TODO: It's pretty weird that we need to set a min_height for the pn.Column
- TODO: I get an error when trying to add a new annotation
- TODO: why is it so slow to change the visibility of the categories? Actually it stopped working altogether
- TODO: color the uncommited rows of the table and have an additional column 'Commited'
- TODO: propagate edits directly on the tabulator table
- TODO: change the GUI buttons to something reasonable.. a 'commit' button should change style if there are uncommitted changes
- TODO: the pan tool is selected but there's seemingly no way to use it while the annotator is active and takes over the click and drag

In [ ]:
annotations_overlay = annotator.get_element("Time").opts(show_legend=False, responsive=True)

curves_annotations = (curves_overlay * annotations_overlay).opts(
    ylabel="Channel",
    show_legend=False,
    responsive=True,
)

app_w_annotations = pn.Column((curves_annotations + minimap * annotations_overlay).cols(1), min_height=800, sizing_mode='stretch_both')
widget_app_annotations = pn.Row(widgets, app_w_annotations)
widget_app_annotations

## Served app

TODO: how to fit the tabulator table in the sidebar of the served app

In [ ]:
template = pn.template.BootstrapTemplate(
    title='Multi-Channel Timeseries App - Time Annotations ',
    main = [app_w_annotations],
    sidebar=[widgets],
)
template.servable();